### Excercise: Day 1 Project using Ollama library with Open Source Model running locally

In [16]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [17]:
# Constants
MODEL = "llama3.2"

In [18]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [19]:
# Let's try one out
site_text = Website("https://vejr.tv2.dk/live/2024-12-02-vejr-og-klima")
print(site_text.title)
print(site_text.text)

Vejr og klima lige nu | Få seneste nyt her - TV 2
Hop til indhold
Nyheder
Sport
Vejr
TV
TV 2 Play
Log ind
Login og indstillinger
Log ind
Opret TV 2 Login
Indstillinger
Farvetema
Log ind for at ændre farvetema
Auto
Vælg farvetema
Lys
Mørk
Auto
Med "Auto" skiftes der automatisk mellem lys og mørk tilstand baseret på din enheds indstillinger.
Nyhedsregion
TV 2 Kosmopol
Vælg nyhedsregion
TV 2 Kosmopol
TV 2 Fyn
tvSyd
TV 2/Bornholm
TV2 ØST
TV2 Østjylland
TV2 Nord
TV Midtvest
Genveje
TV 2 Play
Administrer TV 2 konto
Privatlivsindstillinger
TV 2 privatlivspolitik
Cookie-indstillinger
Menu
Søg, menu og genveje
TV 2 Play
Søg på tv2.dk
Sektioner
Nyheder
Seneste
Korte videoer
Trump i Det Hvide Hus
Krigen i Ukraine
Politik
Krimi
Samfund
Udland
Business
Penge
Tech
Klima
Sport
Seneste
Korte videoer
Sendeplan
Live og resultater
Turneringer
Tour de France
3F Superliga
Tour de France Femmes
Fodbold
Håndbold
Cykling
Badminton
NFL
Tennis
Basketball
Video
Vejr
Seneste
Korte videoer
Vejrudsigt
Radar
Elprise

In [20]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [21]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [22]:
# And now: call the OpenAI API. You will get very familiar with this!
def summarize(url):
    website = Website(url)
    messages=messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [13]:
summarize("https://vejr.tv2.dk/live/2024-12-02-vejr-og-klima")

'Esbjergs Grøn Koncert aflyses på grund af voldsomt vejr'

In [23]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [25]:
display_summary("https://edition.cnn.com/2025/07/24/tech/intel-layoffs-15-percent-q2-earnings")

# Summary of Intel's Staff Reduction and Reorganization Efforts
### Industry: Technology
### Date: July 24, 2025
#### Key Points:
*   Intel Corporation has cut 15% of its workforce, totaling around 33,300 employees, as part of a plan to create a faster-moving, flatter organization.
*   The layoffs are aimed at enhancing the company's competitive position, improving profitability, and creating long-term shareholder value.
*   New CEO Lip-Bu Tan stated that the restructuring is designed to improve the company's agility and competitiveness in the market.

#### Impact:
*   Intel shares rose 3% in after-hours trading following the announcement, indicating investor confidence in Tan's leadership and the company's future prospects.
*   The layoffs are part of a broader cost-cutting effort by Intel, including scrapping projects in Germany and Poland, slowing down construction on Ohio chip factories, and reducing costs by $10 billion.

#### Context:
*   Intel has been struggling to keep pace with rival companies, such as Nvidia, which briefly crossed the $4 trillion market capitalization milestone earlier this month.
*   The company's efforts to play catch-up in the AI chipmaking race have led to a series of bruising years, prompting leadership shakeups and takeover rumors.